In [ ]:
import analysis_toolkit as tk

In [ ]:
context = {
    "file_name": "../data/ser/2021-06-07/03_34_00_pipp.ser",
    "step": 1,
    "fwhm": 22.0,
    "factor": 60.0,
    "aperture_factor_inner": 1.0,
    "aperture_factor_outer": 1.3
}
analyzer = tk.get_default_tk(context)
analyzer.load_analysis()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import matplotlib.dates as mdates
import math

fig_rows, fig_cols = 1, 1
ap_factor = 3.0
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
tk.plot_by_group_lambda_xy(ax, analyzer.objects_apertures_timestamps({analyzer.fwhm * ap_factor}), 
                           tk.x_datetime, lambda table: table[tk.key_ap_factor(ap_factor)]/((analyzer.fwhm * ap_factor)**2*math.pi))

font = {'fontname':'Times New Roman'}
ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fmt_minutes = mdates.MinuteLocator()
ax.xaxis.set_minor_locator(fmt_minutes)
ax.set_xlabel('h(UTC)', fontsize=16, **font)
ax.set_ylabel('flux', fontsize=16, **font)
ax.set_title('J1 ECL J2 07 JUN 2021\nTribsees-T28', fontsize=20, **font)
ax.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure

fig_rows, fig_cols = 1, 1
fac_1, fac_2 = 1.3, 1.8
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
tk.plot_by_group_lambda_xy(ax, analyzer.objects_apertures_timestamps({analyzer.fwhm * fac_1, analyzer.fwhm * fac_2}), 
                           tk.x_datetime, lambda table: tk.local_background(table, fac_1, fac_2))

font = {'fontname':'Times New Roman'}
ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fmt_minutes = mdates.MinuteLocator()
ax.xaxis.set_minor_locator(fmt_minutes)
ax.set_xlabel('h(UTC)', fontsize=16, **font)
ax.set_ylabel('sky background', fontsize=16, **font)
ax.set_title('J1 ECL J2 07 JUN 2021\nTribsees-T28', fontsize=20, **font)
ax.legend()

plt.show()


In [ ]:
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
tk.plot_by_group_lambda_xy(ax, analyzer.objects_apertures_timestamps({analyzer.fwhm * 3.0, analyzer.fwhm * 3.4}), 
                           tk.x_datetime, lambda table: tk.local_background(table, 3.0, 3.4))
# ax.plot_date(analyzer.frames_timestamps["datetime"], analyzer.frames_timestamps["mad_std(img)"])

font = {'fontname':'Times New Roman'}
ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fmt_minutes = mdates.MinuteLocator()
ax.xaxis.set_minor_locator(fmt_minutes)
ax.set_xlabel('h(UTC)', fontsize=16, **font)
ax.set_ylabel('sky background', fontsize=16, **font)
ax.set_title('J1 ECL J2 07 JUN 2021\nTribsees-T28', fontsize=20, **font)
ax.legend()

plt.show()


In [ ]:
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
tk.plot_by_group_lambda_xy(ax, analyzer.objects_apertures_timestamps({analyzer.fwhm * 3.0, analyzer.fwhm * 3.4}), 
                           tk.x_datetime, lambda table: tk.ap_sum_bg_subtracted(table, 3.0, 3.4)/((context["fwhm"]*3.0)**2 * math.pi))

font = {'fontname':'Times New Roman'}
ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fmt_minutes = mdates.MinuteLocator()
ax.xaxis.set_minor_locator(fmt_minutes)
ax.set_xlabel('h(UTC)', fontsize=16, **font)
ax.set_ylabel('light flux of satellite', fontsize=16, **font)
ax.set_title('J1 ECL J2 07 JUN 2021\nTribsees-T28', fontsize=20, **font)
ax.legend()

plt.show()

In [ ]:
import numpy as np

In [ ]:
data_table = analyzer.objects_apertures_timestamps({analyzer.fwhm * 3.0, analyzer.fwhm * 3.4})

In [ ]:
group_by = "object"
series_by_object = data_table.group_by(group_by)
bin_size = 3.0 # seconds

mask_a = series_by_object.groups.keys[group_by] == "a"
series_a = series_by_object.groups[mask_a]
timestamps_a = series_a["timestamp"]
timestamps_a_bin = np.trunc(timestamps_a * 1.0/bin_size * 1E-7)
series_a_grouped = series_a.group_by(timestamps_a_bin)
series_a_grouped["timestamp_bin"] = timestamps_a_bin * bin_size * 1E7
series_a_bin = series_a_grouped.groups.aggregate(np.mean)

mask_b = series_by_object.groups.keys[group_by] == "b"
series_b = series_by_object.groups[mask_b]
timestamps_b = series_b["timestamp"]
timestamps_b_bin = np.trunc(timestamps_b * 1.0/bin_size * 1E-7)
series_b_grouped = series_b.group_by(timestamps_b_bin)
series_b_grouped["timestamp_bin"] = timestamps_b_bin * bin_size * 1E7
series_b_bin = series_b_grouped.groups.aggregate(np.mean)


In [ ]:
print(series_a_bin.keys())

In [ ]:
import datetime
to_datetime = lambda ts: datetime.datetime.min + datetime.timedelta(seconds=ts/1E7)
series_a_bin["datetime_bin"] = list(map(to_datetime, series_a_bin["timestamp_bin"]))
series_b_bin["datetime_bin"] = list(map(to_datetime, series_b_bin["timestamp_bin"]))


In [ ]:
print(series_a_bin)

In [ ]:
import matplotlib.dates as mdates
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
ax.plot_date(series_a_bin["datetime_bin"], tk.ap_sum_bg_subtracted(series_a_bin, 3.0, 3.4)/((context["fwhm"]*3.0)**2 * math.pi), "b+", label="Europe")
ax.plot_date(series_b_bin["datetime_bin"], tk.ap_sum_bg_subtracted(series_b_bin, 3.0, 3.4)/((context["fwhm"]*3.0)**2 * math.pi), "gx", label="Io")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fmt_minutes = mdates.MinuteLocator()
ax.xaxis.set_minor_locator(fmt_minutes)
ax.set_xlabel('h(UTC)', fontsize=16, **font)
ax.set_ylabel('light flux of satellite', fontsize=16, **font)
ax.set_title('J1 ECL J2 07 JUN 2021\nTribsees-T28', fontsize=20, **font)
ax.legend()

plt.show()


In [ ]:
print(set(series_a_bin["datetime_bin"]) - set(series_b_bin["datetime_bin"]))

# Join binned tables 

In [ ]:
series_a_bin["ap_bg_sub_a_3.0_3.4"] = tk.ap_sum_bg_subtracted(series_a_bin, 3.0, 3.4)
series_b_bin["ap_bg_sub_b_3.0_3.4"] = tk.ap_sum_bg_subtracted(series_b_bin, 3.0, 3.4)

In [ ]:
from astropy.table import Table, join

In [ ]:
join_ab_bins = join(series_a_bin, series_b_bin, keys="datetime_bin", join_type="outer")

In [ ]:
export_join_ab_bins = Table([join_ab_bins["datetime_bin"], join_ab_bins["ap_bg_sub_a_3.0_3.4"], join_ab_bins["ap_bg_sub_b_3.0_3.4"]])

In [ ]:
print(export_join_ab_bins.keys())

In [ ]:
export_join_ab_bins.write("test-export.ecsv")

In [ ]:
print(join_ab_bins.keys())

In [ ]:
print(len(join_ab_bins))

In [ ]:
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.plot_date(join_ab_bins["datetime_bin"], join_ab_bins["ap_bg_sub_a_3.0_3.4"]/join_ab_bins["ap_bg_sub_b_3.0_3.4"], "r+")

ax.grid()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fmt_minutes = mdates.MinuteLocator()
ax.xaxis.set_minor_locator(fmt_minutes)
ax.set_xlabel('h(UTC)', fontsize=16, **font)
ax.set_ylabel('relative light flux of Europe', fontsize=16, **font)
ax.set_title('J1 ECL J2 07 JUN 2021\nTribsees-T28', fontsize=20, **font)
ax.legend()

plt.show()


# Normalize, then bin

In [ ]:
data_table["ap_bg_sub_3.0_3.4"] = tk.ap_sum_bg_subtracted(data_table, 3.0, 3.4)

In [ ]:
print(len(data_table))

In [ ]:
object_names = ["a", "b"]
series_by_object = data_table.group_by("object")
mask_a = series_by_object.groups.keys["object"] == object_names[0]
series_a = series_by_object.groups[mask_a]
mask_b = series_by_object.groups.keys["object"] == object_names[1]
series_b = series_by_object.groups[mask_b]
join_ab_frame = join(series_a, series_b, keys=["frame_id"], table_names=object_names)

In [ ]:
print(join_ab_frame.keys())

In [ ]:
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
ax.plot_date(list(map(to_datetime, join_ab_frame["timestamp_a"])), join_ab_frame["ap_bg_sub_3.0_3.4_a"]/join_ab_frame["ap_bg_sub_3.0_3.4_b"], "r-")
plt.show()

In [ ]:
bin_size = 3.0 # seconds

timestamps_a = join_ab_frame["timestamp_a"]
timestamps_a_bin = np.trunc(timestamps_a * 1.0/bin_size * 1E-7)
join_ab_frame_grouped = join_ab_frame.group_by(timestamps_a_bin)
join_ab_frame_bin = join_ab_frame_grouped.groups.aggregate(np.mean)



In [ ]:
print(join_ab_bins.keys())

In [ ]:
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax1: Axes = fig.add_subplot(2, 1, 1)
ax1.grid()
ax1.plot_date(list(map(to_datetime, join_ab_frame_bin["timestamp_a"])), join_ab_frame_bin["ap_bg_sub_3.0_3.4_a"]/join_ab_frame_bin["ap_bg_sub_3.0_3.4_b"], "r-")
ax2: Axes = fig.add_subplot(2, 1, 2)
ax2.grid()
ax2.plot_date(join_ab_bins["datetime_bin"], join_ab_bins["ap_bg_sub_a_3.0_3.4"]/join_ab_bins["ap_bg_sub_b_3.0_3.4"], "y-")
plt.show()

# Look into further quality criteria

In [ ]:
print(analyzer.sources_objects_timestamps.keys())

In [ ]:
print(analyzer.objects_apertures_timestamps({analyzer.fwhm * 3.0, analyzer.fwhm * 3.4}).keys())

In [ ]:
print(analyzer.sources.keys())

In [ ]:
src_ap_3_0_3_4 = join(analyzer.sources, analyzer.objects_apertures_timestamps({analyzer.fwhm * 3.0, analyzer.fwhm * 3.4}))

In [ ]:
print(src_ap_3_0_3_4.keys())

## Add column with corrected aperture sum (background subtracted)

In [ ]:
src_ap_3_0_3_4["ap_bg_sub_3.0_3.4"] = tk.ap_sum_bg_subtracted(src_ap_3_0_3_4, 3.0, 3.4)

In [ ]:
print(src_ap_3_0_3_4.keys())

## Analyse potential correlations *sharpness*/*roundness* vs flux

In [ ]:
table_by_object = src_ap_3_0_3_4.group_by("object")
mask = table_by_object.groups.keys["object"] == "b"
filtered_table = table_by_object.groups[mask]

fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
ax.scatter(filtered_table["roundness1"], filtered_table["roundness2"], 
           c=filtered_table["ap_bg_sub_3.0_3.4"], s=10.0*filtered_table["sharpness"], alpha=0.5)
plt.show()

In [ ]:
fig: Figure = plt.figure(figsize=(9, 9), dpi=300)
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
ax.hist2d(filtered_table["roundness1"], filtered_table["roundness2"], bins=2**5)
plt.show()

In [ ]:
fig_rows, fig_cols = 2, 2
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
    
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
ax.hist(filtered_table["roundness1"], bins=2**6)

ax: Axes = fig.add_subplot(fig_rows, fig_cols, 2)
ax.grid()
ax.hist(filtered_table["roundness2"], bins=2**6)

ax: Axes = fig.add_subplot(fig_rows, fig_cols, 3)
ax.grid()
ax.hist(filtered_table["sharpness"], bins=2**6)

plt.show()

In [ ]:
to_datetime_col = lambda table: np.array(list(map(to_datetime, table["timestamp"])))
ap_bg_sub_3_0_3_4 = lambda table: np.array(table["ap_bg_sub_3.0_3.4"])

In [ ]:
fig_rows, fig_cols = 1, 1
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
    
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
tk.plot_by_group_lambda_xy(ax, src_ap_3_0_3_4, to_datetime_col, ap_bg_sub_3_0_3_4)
plt.show()

In [ ]:
fig_rows, fig_cols = 1, 1
fig: Figure = plt.figure(figsize=(16, 9), dpi=300)
    
src_ap_3_0_3_4_by_sharpness = src_ap_3_0_3_4.group_by("sharpness")
mask_sharpness = src_ap_3_0_3_4_by_sharpness.groups.keys["sharpness"] > 0.5

    
ax: Axes = fig.add_subplot(fig_rows, fig_cols, 1)
ax.grid()
tk.plot_by_group_lambda_xy(ax, src_ap_3_0_3_4_by_sharpness.groups[mask_sharpness], to_datetime_col, ap_bg_sub_3_0_3_4)
plt.show()